In [75]:
import sys
import numpy as np
import mxnet as mx
import tensorflow as tf
from datascience import *
import math as math

In [3]:
# lyrics = Table().read_table('lyrics.csv')
# altered = lyrics.select(np.arange(2, lyrics.num_columns))
# altered.to_csv('genreAndProp.csv')
table = Table().read_table('vectorLyrics.csv')
# propsOnly = lyrics.select(np.arange(3, lyrics.num_columns))
# genresOnly = Table().with_columns('Country', lyrics.apply(lambda x: ((int) (x=='Country')) , 'Genre'), 'Hip-hop', lyrics.apply(lambda x: ((int) (x=='Hip-hop')) , 'Genre'))
# propsOnly.to_csv('lyricsProps.csv')
# genresOnly.to_csv('lyricsGenres.csv')

Our input vectors are of length 4816, because we have 4816 words.
Our output vectors are of length 2, and are a softmax. 1 in the first position means country, 1 in the second position means hip hop.

In [4]:
# filename_queue = tf.train.string_input_producer(["lyricsProps.csv", "lyricsGenres.csv"], num_epochs = 6)

# reader = tf.TextLineReader(skip_header_lines=1)
# key, value = reader.read(filename_queue)

# # Default values, in case of empty columns. Also specifies the type of the
# # decoded result. 
# record_defaults = [[0] for _ in range(4819)]
# reading = tf.decode_csv(value, record_defaults=record_defaults)
# features = tf.stack(reading[0:len(reading)-2])
# genres = tf.stack(reading[len(reading)-2:])

In [5]:
# def create_file_reader_ops(filename_queue):
#     reader = tf.TextLineReader(skip_header_lines=1)
#     _, csv_row = reader.read(filename_queue)
#     record_defaults = [[0] for _ in range(4819)]
#     reading = tf.decode_csv(csv_row, record_defaults=record_defaults)
#     features = tf.stack(reading[0:len(reading)-2])
#     genres = tf.stack(reading[len(reading)-2:])
#     return features, genres

In [6]:
# directory = "vectorLyrics.csv"
# filename_queue = tf.train.string_input_producer(
#     tf.train.match_filenames_once(directory),
#     shuffle=True)

# # Each file will have a header, we skip it and give defaults and type information
# # for each column below.
# line_reader = tf.TextLineReader(skip_header_lines=1)

# _, csv_row = line_reader.read(filename_queue)

# # Type information and column names based on the decoded CSV.
# record_defaults = [[0.0] for _ in range(4817)]+[[0], [0]]
# readings = tf.decode_csv(csv_row, record_defaults=record_defaults)
# # Turn the features back into a tensor.
# features = tf.stack(readings[:4817])
# genre = readings[4817:]

# with tf.Session() as sess:
#     tf.global_variables_initializer().run()

#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)

#     # We do 10 iterations (steps) where we grab an example from the CSV file. 
#     for iteration in range(1, 11):
#         # Our graph isn't evaluated until we use run unless we're in an interactive session.
#         example, label = sess.run([features, genre])

#         print(example, label)
#     coord.request_stop()
#     coord.join(threads)

In [30]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [228]:
# x = tf.placeholder(tf.float32, [None, 4976])
# W1 = tf.Variable(tf.zeros([4976, 1000]))
# b1 = tf.Variable(tf.zeros([1000]))
# y1 = tf.nn.relu(tf.matmul(x, W1) + b1)

# W2 = tf.Variable(tf.zeros([1000, 10]))
# b2 = tf.Variable(tf.zeros([10]))
# y2 = tf.sigmoid(tf.matmul(y1, W2) + b2)

# W3 = tf.Variable(tf.zeros([10, 2]))
# b3 = tf.Variable(tf.zeros([2]))
# y3 = tf.nn.softmax(tf.matmul(y2, W3) + b3)


x = tf.placeholder(tf.float32, [None, 4976])
# W = tf.Variable(tf.zeros([4976, 2]))
W = tf.Variable(tf.truncated_normal([4976, 2],
                                    stddev=1./math.sqrt(4976)))
b = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
# y = tf.transpose(y)

In [229]:
y_ = tf.placeholder(tf.float32, [None, 2])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [186]:
arr = np.array(table.row(0))
trainX = np.array(np.array([arr[:4976]]))
trainY = np.array(np.array([arr[4976:]]))
trainY1 = np.array(np.array([arr[4976:4977]]))
for i in range(1, table.num_rows):
    arr = np.array(table.row(i))
    trainX = np.vstack((trainX, np.array([arr[:4976]])))
    trainY = np.vstack((trainY, np.array([arr[4976:]])))

In [230]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
count = 0
while count < 10:
    try:
        sample = table.sample(100)
        arr = np.array(sample.row(0))
        features = np.array(np.array([arr[:4976]]))
        genre = np.array(np.array([arr[4976:]]))
        for i in range(1, 100):
            arr = np.array(table.row(i))
            features = np.vstack((features, np.array([arr[:4976]])))
            genre = np.vstack((genre, np.array([arr[4976:]])))
#         features = np.array([arr[:4976]])
#         genre = np.array([arr[4976:]])
        feed_dict={x: features, y_: genre}
        example_data = sess.run(train_step, feed_dict)
#         print(genre)
#         print(example_data)
    except tf.errors.OutOfRangeError:
        break
    print('training session' + str(count))
    count+=1

training session0
training session1
training session2
training session3
training session4
training session5
training session6
training session7
training session8
training session9


In [221]:
correct_prediction = tf.equal(tf.round(y), tf.round(y_))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: trainX , y_: trainY }))

0.511912


In [222]:
holdout = Table.read_table('holdout.csv')
arr = np.array(table.row(0))
testX = np.array(np.array([arr[2:]]))
for i in range(1, holdout.num_rows):
    arr = np.array(holdout.row(i))
    testX = np.vstack((testX, np.array([arr[2:]])))

KeyboardInterrupt: 

In [223]:
hCoun = np.array([])
hHip = np.array([])
prediction=tf.argmax(y,1)
print ("predictions", prediction.eval(feed_dict={x: trainX}, session=sess))

predictions [ 83 916]


In [224]:
probabilities=y
print ("probabilities", probabilities.eval(feed_dict={x: testX}, session=sess))

probabilities [[ 0.95281273  0.95495874  0.93714488  0.95791137  0.95415217  0.95960534
   0.95838273  0.95817035  0.95360094  0.96467274  0.95318395  0.95921332
   0.96434861  0.95730191  0.9493764   0.95143819  0.95944774  0.96125787
   0.95159203  0.94981152  0.95166981  0.95455039  0.9500885   0.95970988
   0.96403033  0.95349646  0.92748415  0.95893437  0.96197242  0.95644301
   0.95402086  0.95746231  0.95380652  0.95056307  0.95169479  0.95363355
   0.95037043  0.95816344  0.96641344  0.95268452  0.95647687  0.95249194
   0.95914102  0.95683122  0.96154886  0.95891374  0.95886999  0.96019781
   0.96029788  0.95449948  0.95537806  0.95409298  0.95421606  0.92876279
   0.95388979  0.94038534  0.94978118  0.95288998  0.93619901  0.95234966
   0.95644629  0.9538818   0.95694143  0.95691448]
 [ 0.04718723  0.04504126  0.06285508  0.04208861  0.04584791  0.04039467
   0.04161731  0.04182965  0.04639909  0.03532731  0.04681605  0.04078669
   0.03565146  0.04269804  0.05062366  0.048561

In [64]:
table

i         | the       | you       | to         | and        | a          | me        | it         | not       | in         | my         | is         | of         | your       | that       | do         | on         | are        | we         | am         | will       | all        | for        | no         | be         | have       | love       | so         | know       | this       | but        | with       | what       | just       | when      | like       | now        | que       | time       | can        | come       | de        | there      | go         | up         | oh         | la       | one        | they       | out        | down       | get        | she        | was        | see        | if         | got        | never      | from       | he         | feel       | want       | let        | make       | way        | say        | take       | would      | as         | ca         | day        | at         | babi       | away       | life       | yeah       | y        | back       | by         | her        | heart      | here       | how        | could      | night      | need       | our        | look       | where      | en         | eye  | thing      | world      | more       | caus       | gonna      | die        | right      | been       | tell       | think     | un   | who        | el        | through    | man        | live       | again      | give       | too        | onli       | te       | tri        | tu   | or         | whi        | se   | keep       | dream      | well       | mind       | an         | wo         | still      | us         | his        | long       | mi   | girl       | wanna      | find       | fall      | around     | good       | about      | some       | over       | littl      | turn       | hand       | then       | noth       | light      | said       | call       | everi      | alway      | were       | did        | into       | e    | wait | leav       | home | had        | hold | run        | es         | walk       | ever       | them       | lie       | face     | end        | hear       | gone       | head       | believ     | everyth | ya         | has  | cri       | alon       | yo         | hey        | their      | le   | someth     | si        | insid | du   | befor      | o    | word       | stay       | place      | much       | chang      | ich  | god        | stand      | these      | better     | last       | friend | soul | anoth      | je   | left       | new        | him        | than       | stop       | tonight | burn | sun        | off        | por      | lo         | play       | thought    | sing | show       | lost | peopl      | made       | break      | und  | start      | should     | realli     | chorus | hard | sky  | old        | da   | even | boy  | pleas      | song       | care       | without | hope | watch      | pain       | amor     | move       | wrong      | put        | free       | gotta      | name | et   | same       | blue | con      | danc | talk       | seem | own  | mine       | dark      | na   | far  | tear | true | fire       | use        | dead | bring      | other | sin      | someon | myself | forev | close     | sleep | rain       | those      | must | una  | les  | sweet      | mean       | fight | rememb     | star       | while      | kiss | kill       | high | breath | doe        | yes  | al   | wish       | che  | real       | smile      | two        | cold       | lord | year       | open       | il   | fear       | di   | done       | side       | fli  | black      | line       | rock       | blood      | lose | found     | onc        | non  | door | bad  | ooh  | wonder | ti   | como       | enough     | para       | sound | togeth | mayb       | los        | help       | shine      | mani       | heaven     | becaus     | work       | touch | behind     | yourself | today      | big  | might | death | hate       | miss | ride       | street     | came       | ask        | hide       | everybodi | save       | till | may  | 